# Star Multi-Table Dataset

In this notebook, we will learn how to train a classifier with a multi-table data composed of two tables (a root table and a secondary table). It is highly recommended to see the _Sklearn Basics 1_ lesson if you are not familiar with pyKhiops' sklearn estimators.


We start by importing pyKhiops sklearn classifier `KhiopsClassifier` and saving the location of the Khiops `Samples` directory into a variable:

In [ ]:
from os import path
import pandas as pd

from khiops import core as kh
from khiops.sklearn import KhiopsClassifier

### Training a Multi-Table Classifier
We'll train a "sarcasm detector" using the dataset `HeadlineSarcasm`. In its raw form, the dataset contains a list of text headlines paired with a label that indicates whether its source is a sarcastic site (such as [The Onion](https://www.theonion.com)) or not.

We have transformed this dataset into two tables such that the text-label record
```
"groundbreaking study finds gratification can be deliberately postponed"	yes
```
is transformed to an entry in a table that contains (id, label) records
```
97 yes
```
and various entries in a secondary table linking a headline id to its words and positions
```
97	0	groundbreaking
97	1	study
97	2	finds
97	3	gratification
97	4	can
97	5	be
97	6	deliberately
97	7	postponed
```

Thus, the `HeadlineSarcasm` dataset has the following multi-table schema
```
 +-----------+
 |Headline   |
 +-----------+ +-------------+
 |HeadlineId*| |HeadlineWords|
 |IsSarcastic| +-------------+
 +-----------+ |HeadlineId*  |
      |        |Position     |
      +-1:n--->|Word         |
               +-------------+
```
The `HeadlineId` variable is special because it is a _key_ that links a particular headline to its words (a `1:n` relation).

*Note: There are other methods more appropriate for this text-mining problem. This multi-table setup is only intended for pedagogical purposes.*

To train the `KhiopsClassifier` for this setup we must specify a multi-table dataset. Let's first check the content of the created tables: 
- The main table `Headline`
- The secondary table `HeadlineWords`

In [ ]:
sarcasm_dataset_dir = path.join("data", "HeadlineSarcasm")
headlines_file = path.join(sarcasm_dataset_dir, "Headlines.txt")
headlines_df = pd.read_csv(headlines_file, sep="\t")
print("Headlines table (first 10 rows)")
display(headlines_df.head(10))

headlines_words_file = path.join(sarcasm_dataset_dir, "HeadlineWords.txt")
headlines_words_df = pd.read_csv(headlines_words_file, sep="\t")
print("HeadlineWords table (first 10 rows)")
display(headlines_words_df.head(10))

Before training the classifier, we split the main table into a feature matrix (only the `HeadlineId` column) and a target vector containing the labels (the `IsSarcasm` column).

In [ ]:
headlines_train_df = headlines_df.drop("IsSarcasm", axis=1)
y_sarcasm_train = headlines_df["IsSarcasm"]

You may note that the feature matrix does not contain any *feature* but do not worry. The Khiops AutoML engine will automatically create features by aggregating the columns of `HeadlineWords` for each headline (more details about this below). 

Moreover, instead of passing an `X` table to the `fit` method, we pass a *multi-table dataset* specification which is dictionary with the following format:
```
X = {
   "main_table": <name of the main table>,
   "tables" : {
       <name of table 1>: (<dataframe of table 1>, <key column names of table 1>),
       <name of table 2>: (<dataframe of table 2>, <key column names of table 2>),
       ...
    }
}
```
Note that the key columns of each table are specified as a single name or a tuple containing the column names composing the key.

So for our `HeadlineSarcasm` case, we specify the dataset as:

In [ ]:
X_sarcasm_train = {
    "main_table": "headlines",
    "tables": {
        "headlines": (headlines_train_df, "HeadlineId"),
        "headline_words": (headlines_words_df, "HeadlineId"),
    },
}

The call to the `KhiopsClassifier` `fit` method is very similar to the single-table case but this time we specify the additional parameter `n_features` which is the number of aggregates that Khiops AutoML engine will construct and analyze during the training. Some examples of the features it will create for `HeadlineSarcasm` are:
- Number of different words in the headline
- Most common word in the headline
- Number of times the word 'the' appears
- ...

The Khiops AutoML engine will also evaluate, select and combine the these features to build a classifier. We'll here request for `1000` features (the default is `100`):

*Note: By default Khiops builds 10 decision tree features. This is not necessary for this tutorial so we set `n_trees=0`*

In [ ]:
pkc_sarcasm = KhiopsClassifier(n_features=1000, n_trees=0)
pkc_sarcasm.fit(X_sarcasm_train, y_sarcasm_train)

We quickly check its train accuracy and auc as in the previous tutorial:

In [ ]:
sarcasm_train_performance = (
    pkc_sarcasm.model_report_.train_evaluation_report.get_snb_performance()
)
print(f"HeadlineSarcasm train accuracy: {sarcasm_train_performance.accuracy}")
print(f"HeadlineSarcasm train auc     : {sarcasm_train_performance.auc}")

Now, we use our sarcasm classifier to obtain predictions on the training data. We normally do that on new test data, and again a multi-table dataset specification would have been needed.

In [ ]:
sarcasm_predictions = pkc_sarcasm.predict(X_sarcasm_train)
print("HeadlineSarcasm train predictions (first 10 values):")
display(sarcasm_predictions[:10])

### Exercise

Repeat the previous steps with the `AccidentsSummary` dataset. This dataset describes the characteristics of traffic accidents that happened in France in 2018. It has two tables with the following schema:
```
+---------------+
|Accidents      |
+---------------+
|AccidentId*    |
|Gravity        |
|Date           |
|Hour           | +---------------+
|Light          | |Vehicles       |
|Department     | +---------------+
|Commune        | |AccidentId*    |
|InAgglomeration| |VehicleId*     |
|...            | |Direction      |
+---------------+ |Category       |
       |          |PassengerNumber|
       +---1:n--->|...            |
                  +---------------+
```
For each accident, we have both its characteristics (such as `Gravity` or `Light` conditions) and those of each involved vehicle (its `Direction` or `PassengerNumber`). We first load the tables of the `AccidentsSummary` into dataframes:

In [ ]:
accidents_dataset_dir = path.join(kh.get_samples_dir(), "AccidentsSummary")

accidents_file = path.join(accidents_dataset_dir, "Accidents.txt")
accidents_df = pd.read_csv(accidents_file, sep="\t", encoding="ISO-8859-1")
print(f"Accidents dataframe (first 10 rows):")
display(accidents_df.head(10))
print()

vehicles_file = path.join(accidents_dataset_dir, "Vehicles.txt")
vehicles_df = pd.read_csv(vehicles_file, sep="\t", encoding="ISO-8859-1")
print(f"Vehicles dataframe (first 10 rows):")
display(vehicles_df.head(10))

#### Create the main feature matrix and the target vector for `AccidentsSummary`

Note that the target variable is `Gravity`.

In [ ]:
accidents_main_df = accidents_df.drop("Gravity", axis=1)
y_accidents_train = accidents_df["Gravity"]

#### Create the multi-table dataset specification

Note the main table has one key `AccidentId` and the secondary table has two keys `AccidentId` and `VehicleId`.

In [ ]:
X_accidents_train = {
    "main_table": "accidents",
    "tables": {
        "accidents": (accidents_main_df, "AccidentId"),
        "vehicles": (vehicles_df, ["AccidentId", "VehicleId"]),
    },
}

#### Train a classifier with this dataset

- You may choose the number of features `n_features` to be created by the Khiops AutoML engine
- Set the number of trees to zero (`n_trees=0`)

In [ ]:
pkc_accidents = KhiopsClassifier(n_trees=0, n_features=1000)
pkc_accidents.fit(X_accidents_train, y_accidents_train)

#### Print the accuracy and auc of the model


In [ ]:
accidents_train_performance = (
    pkc_accidents.model_report_.train_evaluation_report.get_snb_performance()
)
print(f"AccidentsSummary train accuracy: {accidents_train_performance.accuracy}")
print(f"AccidentsSummary train auc     : {accidents_train_performance.auc}")

#### Deploy the classifier to obtain predictions on the training data

*Note that usually one deploys the model on new test data. We deploy on the train dataset to keep the tutorial simple*.


In [ ]:
pkc_accidents.predict(X_accidents_train)